In [4]:
import tkinter as tk
from tkinter import scrolledtext
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import datetime

responses = {
    "hours": "Our customer service is available 9 AM - 6 PM, Monday to Saturday.",
    "refund": "You can apply for a refund within 7 days of purchase.",
    "track order": "Please provide your order ID to check the status.",
    "greeting": "Hello! How can I assist you today?",
    "goodbye": "Thank you for contacting us. Have a great day!",
    "payment": "We accept all major credit cards, debit cards, and online payment options.",
    "support": "You can contact our support team via email or phone.",
}

keywords = {
    "greeting": ["hello", "hi", "hey", "good morning", "good afternoon"],
    "hours": ["hours", "timing", "open", "close"],
    "refund": ["refund", "return", "money back"],
    "track order": ["track", "order", "status", "shipment"],
    "goodbye": ["bye", "exit", "thank you", "thanks"],
    "payment": ["payment", "pay", "credit", "debit"],
    "support": ["support", "help", "contact", "assistance"],
}

# Preprocessing function
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Conversation logging
def log_conversation(user_input, bot_response):
    with open("conversation_log.txt", "a") as f:
        f.write(f"{datetime.datetime.now()} | You: {user_input} | Bot: {bot_response}\n")

# State variable for interactive order ID
waiting_for_order_id = False

# Bot response function
def get_bot_response(user_input):
    global waiting_for_order_id
    clean_input = preprocess(user_input)

    # If waiting for order ID
    if waiting_for_order_id:
        order_id = clean_input
        waiting_for_order_id = False
        response = f"Your order {order_id} is being processed. Expected delivery: 3 days."
        log_conversation(user_input, response)
        return response, False

    # Rule-based keyword matching
    for key, kw_list in keywords.items():
        if any(word in clean_input for word in kw_list):
            if key == "track order":
                waiting_for_order_id = True
            log_conversation(user_input, responses[key])
            return responses[key], key == "goodbye"

    # NLP similarity fallback
    training_data = list(responses.keys())
    vectorizer = TfidfVectorizer().fit_transform(training_data)
    user_vec = TfidfVectorizer().fit(training_data).transform([clean_input])
    similarities = cosine_similarity(user_vec, vectorizer).flatten()
    best_match_idx = similarities.argmax()

    if similarities[best_match_idx] > 0:
        best_match = training_data[best_match_idx]
        log_conversation(user_input, responses[best_match])
        return responses[best_match], False
    else:
        fallback = "I'm sorry, I didn’t understand that. Could you rephrase?"
        log_conversation(user_input, fallback)
        return fallback, False

# GUI Setup
def send_message():
    user_input = user_entry.get()
    if not user_input.strip():
        return
    chat_window.config(state='normal')
    chat_window.insert(tk.END, "You: " + user_input + "\n")
    user_entry.delete(0, tk.END)

    bot_response, exit_flag = get_bot_response(user_input)
    chat_window.insert(tk.END, "Bot: " + bot_response + "\n\n")
    chat_window.config(state='disabled')
    chat_window.yview(tk.END)

    if exit_flag:
        root.after(1000, root.destroy)

root = tk.Tk()
root.title("Customer Service Chatbot")

chat_window = scrolledtext.ScrolledText(root, wrap=tk.WORD, state='disabled', width=60, height=20)
chat_window.pack(padx=10, pady=10)

user_entry = tk.Entry(root, width=50)
user_entry.pack(side=tk.LEFT, padx=(10,0), pady=(0,10))

send_button = tk.Button(root, text="Send", command=send_message)
send_button.pack(side=tk.LEFT, padx=10, pady=(0,10))

chat_window.config(state='normal')
chat_window.insert(tk.END, "🤖 Hello! Welcome to our Customer Service Chatbot.\nType your query below.\n\n")
chat_window.config(state='disabled')

root.mainloop()
